In [ ]:
!pip install -q openai-whisper moviepy

In [ ]:
import whisper
from moviepy.editor import *
import os
import json
import numpy as np
import pandas as pd
import joblib

  if event.key is 'enter':



In [1]:
data_dir = "/path/to/data"

# Transcribe

## Whisper

In [ ]:
model = whisper.load_model('large-v2')

100%|██████████████████████████████████████| 2.87G/2.87G [00:26<00:00, 114MiB/s]
  checkpoint = torch.load(fp, map_location=device)



In [ ]:
result = model.transcribe(os.path.join(data_dir, 'encounter.m4v'))

In [ ]:
type(result)

dict

In [ ]:
with open(os.path.join(data_dir, 'video_transcription.json'), 'wt', encoding='utf-8') as f:
  json.dump(result, f)

## Speaker Diarization

In [ ]:
def whisper_results_to_srt(whisper_results):
    srt_content = ""
    for i, segment in enumerate(whisper_results['segments'], start=1):
        # Convert start and end times from seconds to SRT timestamp format
        start_time = format_timestamp_srt(segment['start'])
        end_time = format_timestamp_srt(segment['end'])
        text = segment['text']

        # Append formatted segment to SRT content
        srt_content += f"{i}\n{start_time} --> {end_time}\n{text}\n\n"

    return srt_content

In [ ]:
def format_timestamp_srt(seconds):
    """Converts seconds to SRT timestamp format (HH:MM:SS,mmm)"""
    hours, remainder = divmod(seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    milliseconds = (seconds - int(seconds)) * 1000
    return f"{int(hours):02}:{int(minutes):02}:{int(seconds):02},{int(milliseconds):03}"

In [ ]:
srt_content = whisper_results_to_srt(result)

In [ ]:
# Input data
srt_file = "video.srt"

In [ ]:
with open(srt_file, 'wt', encoding='utf-8') as file:
    file.write(srt_content)

In [ ]:
!cp srt_file "{data_dir}"

**GPT-4 prompt**

In [ ]:
!pip install -q langchain-community langchain-openai langchain-anthropic

In [ ]:
#os.environ['OPENAI_API_KEY'] = ''
#os.environ['ANTHROPIC_API_KEY'] = ''

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai.chat_models import ChatOpenAI
from langchain_anthropic.chat_models import ChatAnthropic
from langchain_core.output_parsers import StrOutputParser
from pydantic import BaseModel, Field

In [ ]:
example = """
## HERE IS THE EXAMPLE ##
Srt file input is enclosed in backticks:
```
1
00:00:00,000 --> 00:00:01,500
 My name is Dr. House.

2
00:00:01,500 --> 00:00:02,700
 And this is Nelson, correct?

3
00:00:02,700 --> 00:00:03,200
 Yes.

4
00:00:03,200 --> 00:00:04,500
 Very nice to meet you.

5
00:00:04,500 --> 00:00:05,500
 Nice to meet you.

6
00:00:05,500 --> 00:00:11,500
 I understand I read outside some information that you're here for your son, who's 26 months old.```

ANSWER:
1. Doctor
2. Doctor
3. Patient
4. Doctor
5. Patient
6. Doctor
"""

In [ ]:
user_message = """You are a helpful assistant for medical english and doctor patient communication.
I will provide a transcript text (in the form of srt file) from a doctor patient encounter recorded in an OSCE exam.
However, this transcript does not separate the doctor and the patient.
Can you please identify the speaker in each transcript segment as doctor or patient?
""" + example + """\nCan you please evaluate the following srt: \n{srt}"""

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("user", user_message)
])

In [ ]:
_template = """Can you please inspect this input and convert it to a python list for numbers realted to doctor and patient like in the example below?.
JUST GIVE THE PYTHON LISTS AND NOTHING ELSE!
## Example ##
INPUT:
1. Doctor
2. Doctor
3. Patient
4. Doctor
5. Patient
6. Doctor
OUTPUT:
doctor = [1, 2, 4, 6]
patient = [3, 5]

INPUT:
{text}
OUTPUT:
"""
evalpr = ChatPromptTemplate.from_template(_template)
eval = evalpr | ChatOpenAI(temperature=0, model="gpt-4o-mini")

In [ ]:
from typing import List

In [ ]:
class DoctorPhrases(BaseModel):
    """Extracts the number in srt content that belongs to doctor in a doctor patient talk."""
    lst : List = Field(..., description="List of numbers in the srt file content that belongs to the doctor in a doctor patient encounter transcription.")

In [ ]:
chain = prompt | ChatOpenAI(temperature=0, model="gpt-4o").bind_tools([DoctorPhrases], tool_choice='DoctorPhrases')

In [ ]:
from langchain_anthropic.chat_models import ChatAnthropic

In [ ]:
chain1 = prompt | ChatAnthropic(temperature=0, model="claude-3-5-sonnet-20241022").bind_tools([DoctorPhrases], tool_choice='DoctorPhrases')

In [ ]:
def parse_srt(file_path):
    chunks = []
    with open(file_path, 'r') as file:
        chunk = []
        for line in file:
            line = line.strip()
            if line:  # Non-empty line
                chunk.append(line)
            else:  # Empty line indicates the end of a chunk
                if chunk:
                    chunks.append("\n".join(chunk))
                    chunk = []
        # Append the last chunk if the file doesn't end with an empty line
        if chunk:
            chunks.append("\n".join(chunk))
    return chunks

In [ ]:
srt_file = "video.srt"
srt_file = '/content/gdrive/MyDrive/MLprojects/osce_pipeline3/data/video4/video.srt'
chunks = parse_srt(srt_file)

In [ ]:
res = chain.invoke({'srt':'\n'.join(chunks[:10])})

In [ ]:
res

## Update SRT

In [ ]:
def add_speaker_prefix_old(srt_file, doctor_indices, patient_indices, output_file):
    with open(srt_file, 'r') as file:
        lines = file.readlines()

    updated_lines = []
    i = 0

    while i < len(lines):
        line = lines[i].strip()

        if line.isdigit():  # Start of a new SRT block (ID line)
            index = int(line)
            updated_lines.append(line)  # Add the ID line
            i += 1

            # Add the timestamp line
            updated_lines.append(lines[i].strip())
            i += 1

            # Add text with the speaker prefix
            text_lines = []
            while i < len(lines) and lines[i].strip():
                text_lines.append(lines[i].strip())
                i += 1

            if index in doctor_indices:
                text_lines = [f"Doctor: {text}" for text in text_lines]
            elif index in patient_indices:
                text_lines = [f"Patient: {text}" for text in text_lines]

            updated_lines.extend(text_lines)
        else:
            updated_lines.append(line)
        i += 1

    # Write the updated SRT to the output file
    with open(output_file, 'w') as file:
        file.write("\n".join(updated_lines) + "\n")

In [ ]:
def add_speaker_prefix(srt_file, doctor_indices, output_file):
    with open(srt_file, 'r') as file:
        lines = file.readlines()

    updated_lines = []
    i = 0

    while i < len(lines):
        line = lines[i].strip()

        if line.isdigit():  # Start of a new SRT block (ID line)
            index = int(line)
            updated_lines.append(line)  # Add the ID line
            i += 1

            # Add the timestamp line
            updated_lines.append(lines[i].strip())
            i += 1

            # Collect text lines
            text_lines = []
            while i < len(lines) and lines[i].strip():
                text_lines.append(lines[i].strip())
                i += 1

            # Add speaker prefix
            if index in doctor_indices:
                text_lines = [f"Doctor: {text}" for text in text_lines]
            else:  # Patient by default
                text_lines = [f"Patient: {text}" for text in text_lines]

            updated_lines.extend(text_lines)
        else:
            updated_lines.append(line)
        i += 1

    # Write the updated SRT to the output file
    with open(output_file, 'w') as file:
        file.write("\n".join(updated_lines) + "\n")


In [ ]:
def iterate_in_chunks_with_overlap(lst, chunk_size, window_size):
    step = chunk_size - window_size  # Define the step to maintain overlap
    if step <= 0:
        raise ValueError("Chunk size must be greater than window size to create overlap.")

    for i in range(0, len(lst) - window_size + 1, step):
        yield lst[i:i + chunk_size]

In [ ]:
chunk_size = 30  # Define the size of each chunk
window_size = 5  # Define the overlap size

In [ ]:
doctor_lst = []

In [ ]:
for chunk in iterate_in_chunks_with_overlap(chunks, chunk_size, window_size):
    res = chain.invoke({'srt': '\n'.join(chunk)})
    lst = res.tool_calls[0]['args']['lst']
    doctor_lst.extend(lst)
    print(lst)

[1, 2, 8, 9, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 27, 28, 29]
[28, 29, 30, 31, 32, 34, 36, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54]
[52, 53, 54, 55, 56, 57, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80]
[76, 77, 78, 79, 80, 81, 82, 83, 84, 87, 88, 89, 90, 91, 92, 93, 94, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105]
[101, 102, 103, 103, 104, 104, 104, 105, 106, 106, 107, 108, 108, 109, 110, 111, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130]
[126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136]


In [ ]:
doctor = sorted(list(set(doctor_lst)))

In [ ]:
output_file = "video_updated2.srt"

# Update the SRT file
add_speaker_prefix(srt_file, doctor, output_file)

In [ ]:
!cp "{output_file}" "{data_dir}"

# Verbal

In [ ]:
verbal = ['Reaches Agreement', 'Shares Information', 'Builds a Relationship',
          'Shows Verbal Empathy', 'Provides Structure', 'Gathers Information',
          'Provides Closure', "Understands the Patient's Perspective",
          'Opens the Discussion', 'Use of Fillers vs Silence']

In [ ]:
overall = ['English Proficiency', 'Overall Impression',
           'Use of Facial Expression', 'Use of Body Language', ]

In [ ]:
class BuildsRelationship(BaseModel):
    """
    Greets and shows interest in patient as a person
    uses words that show care, respect, and non-judgment throughout interview
    asks for permission (e.g., before asking sensitive questions or during physical examination)
    remains attentive and professional"""

    score: int = Field(
        ...,
        description="Describes whether relationship building skills are used or not.",
        enum=[0, 1],
    )

In [ ]:
class OpensDiscussion(BaseModel):
    """
    Allows patient to complete opening statement(s) without interruption
    asks “Is there anything else?” (or similar) to elicit full list of concerns
    explains and/or negotiates an agenda for rest of visit
    """

    score: int = Field(
        ...,
        description="Describes whether opening discussion skills are used or not.",
        enum=[0, 1],
    )

In [ ]:
class GathersInformation(BaseModel):
    """Elicits patient story using open-ended questions (e.g., “Tell me about…”)
    attentively listens and clarifies details as necessary with specific (“closed”) questions
    explored the problem like a natural conversation as opposed to an interrogation
    summarizes and gives patient opportunity to correct or add information
    avoids or clearly explains any medical jargon"""

    score: int = Field(
        ...,
        description="Describes whether information gathering skills are used or not.",
        enum=[0, 1],
    )

In [ ]:
class UnderstandsPatientPerspective(BaseModel):
    """
    Elicits patient’s ideas, concerns, and expectations of the illness and treatment
    explores how the problem affects the patient’s life,
    encourages patient to express ideas and feelings and responds effectively
    """
    score: int = Field(
        ...,
        description="Describes whether understanding patient's perspective skills are used or not.",
        enum=[0, 1],
    )

In [ ]:
class ShowsVerbalEmpathy(BaseModel):
    """
    Shows genuine interest in and concern for patient’s situation
    identifies/labels/validates patient’s feelings and emotions
    responds appropriately to patient’s emotional verbal cues
    """
    score: int = Field(
        ...,
        description="Describes whether showing verbal skills are used or not.",
        enum=[0, 1],
    )

In [ ]:
class SharesInformation(BaseModel):
    """
    Assesses patient’s prior knowledge and understanding of the problem and desire for more information
    explains in small “chunks” using words that patient can understand
    repeats information as necessary and checks for understanding
    provides recommendations and/or pros/cons/risks as appropriate,
    encourages patient to ask questions or bring up concerns
    """
    score: int = Field(
        ...,
        description="Describes whether sharing information skills are used or not.",
        enum=[0, 1],
    )

In [ ]:
class ReachesAgreement(BaseModel):
    """
    Includes patient in decision-making to extent s/he desires
    asks about patient’s ability to follow diagnostic and/or treatment plans
    ensures mutual understanding of next steps and/or treatment plan provides \
    support by sharing additional resources as appropriate
    """
    score: int = Field(
        ...,
        description="Describes whether reaching to agreement skills are used or not.",
        enum=[0, 1],
    )

In [ ]:
class ProvidesClosure(BaseModel):
    """
    Asks whether patient has any more questions, concerns, or other issues
    summarizes / asks patient to summarize plan and next steps
    clarifies any follow-up or contact arrangements
    acknowledges patient and closes interview effectively
    """
    score: int = Field(
        ...,
        description="Describes whether providing closure skills are used or not.",
        enum=[0, 1],
    )

In [ ]:
class ProvidesStructure(BaseModel):
    """
    Conducts the interview in a clear, logical, structured way using signposting\
     and transition statements to different lines of questions or parts of interview
    manages time and pace effectively so that consultation does not feel rushed\
     or incomplete, nor affects rapport with patient
    explains what will happen next and rationale if needed
    """
    score: int = Field(
        ...,
        description="describes whether providing structure skills are used or not.",
        enum=[0, 1],
    )

In [ ]:
tools = [BuildsRelationship,
GathersInformation,
OpensDiscussion,
ProvidesClosure,
ProvidesStructure,
ReachesAgreement,
SharesInformation,
ShowsVerbalEmpathy,
UnderstandsPatientPerspective]

In [ ]:
llm = ChatAnthropic(model='claude-3-5-sonnet-20241022', temperature=0)

In [ ]:
human= """Available tiers for this task include:
<available_tiers>
Reaches Agreement, Shares Information, Builds a Relationship, Shows Verbal Empathy, \
Provides Structure, Gathers Information, Provides Closure, Understands the Patient's Perspective, \
Opens the Discussion, Use of Fillers vs Silence
</available_tiers>

For each input (a phrase from doctor's talk), 1) select suitable tier(s) 2) give a binary score for selected each tier."""

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an expert in medical communication skills. You will be annotating verbal communication skills in a doctor patient encounter."),
    ('human', human),
    ("human", "{input}"),
])

In [ ]:
chain2 = prompt | llm.bind_tools(tools)

In [ ]:
result = chain.invoke({"input":'I understand your concerns'})

In [ ]:
result.tool_calls

[{'name': 'ShowsVerbalEmpathy',
  'args': {'score': 1},
  'id': 'toolu_01W9tE1NDa1eAwZ1X5ydJeA8',
  'type': 'tool_call'}]

In [ ]:
print(r.tool_calls[0])
print(r.tool_calls[1])
print(r.tool_calls[2])

{'name': 'ProvidesStructure', 'args': {'score': 1}, 'id': 'toolu_01X7Jwb5zPa3nP7jcifzn558', 'type': 'tool_call'}
{'name': 'ProvidesClosure', 'args': {'score': 1}, 'id': 'toolu_01AsKkKmWXg3LDAPaq9TnB97', 'type': 'tool_call'}
{'name': 'SharesInformation', 'args': {'score': 1}, 'id': 'toolu_01RTzvs3EsWsFDWfESAWBPer', 'type': 'tool_call'}


In [ ]:
def iterate_with_overlap(lst, chunk_size, window_size):
    step = chunk_size - window_size  # Calculate the step size
    if step <= 0:
        raise ValueError("Chunk size must be greater than window size for overlap.")

    for i in range(0, len(lst), step):
        yield i, lst[i:i + chunk_size]  # Yield the starting index and chunk

In [ ]:
def label_phrase(phrase):
    result = chain2.invoke({"input": phrase})
    l = []
    for i in result.tool_calls:
        l.append(i['name'])
    phrase = f"{phrase} [Label: {l}]"
    return phrase

In [ ]:
chunk_size = 50  # Size of each chunk
window_size = 5   # Overlap size

In [ ]:
srt_text = ''

In [ ]:
for start_idx, chunk in iterate_with_overlap(chunks, chunk_size, window_size):
    # Pass the chunk to the chain to get indices of doctor phrases relative to the original list
    res = chain1.invoke({'input': '\n'.join(chunk)})
    doctor_indices = res.tool_calls[0]['args']['lst']  # Indices of doctor phrases in the original list

    # Apply the labeling function only to doctor phrases
    labeled_phrases = []
    for idx, phrase in enumerate(chunk):
        global_idx = start_idx + idx  # Map chunk index to original list index
        if global_idx in doctor_indices:  # Check if the global index is a doctor phrase
            try:
                labeled_ph = label_phrase(phrase)
            except:
                labeled_ph = phrase
            labeled_phrases.append(labeled_ph)
        else:
            labeled_phrases.append(phrase)  # Keep the original phrase if not a doctor phrase

    # Combine labeled phrases into the `srt` format
    srt = '\n'.join(labeled_phrases)
    srt_text += srt + '\n'

In [ ]:
with open('labeled.srt', 'wt', encoding='utf-8') as f:
    f.write(srt_text)